In [1]:
import numpy as np
import pandas as pd

In [2]:
range(201)

range(0, 201)

In [6]:
df = pd.DataFrame(pd.read_table("./data/glove.6B.200d.txt", header=None, names=map(str,range(201)), sep=' '))
#df = pd.DataFrame(pd.read_csv("./data/minimal.50d.3f.csv")).set_index("0")

In [7]:
df = df.set_index("0")

In [8]:
words = df.index

In [9]:
lens = (df**2).sum(axis=1)
dfn = df.div(np.sqrt(lens), axis='index')

In [10]:
def find_most_similar(word, data, n=5):
    dotted = data.dot(data.loc[word])
    return dotted.reindex(dotted.sort_values(ascending = False).index).head(n)

In [11]:
def reject(row, remove, data):
    row = data.loc[row]
    for val in remove:
        val = data.loc[val]
        row = row - (row.dot(val)*val/val.dot(val))
    return row

# Vector rejection

In [12]:
find_most_similar('rock', dfn, 10)

0
rock      1.000000
band      0.694770
pop       0.674067
punk      0.661065
bands     0.645949
'n'       0.624487
rocks     0.616312
album     0.613320
albums    0.600090
music     0.599088
dtype: float64

In [13]:
dfn.dot(reject('rock', ['music'], dfn)).sort_values(ascending = False).head(10)

0
rock           0.641093
rocks          0.498883
outcropping    0.444147
limestone      0.431764
outcrops       0.419863
outcrop        0.409561
cliffs         0.404085
fraggle        0.400296
rockers        0.399462
granite        0.399351
dtype: float64

In [14]:
find_most_similar('tie', dfn, 10)

0
tie      1.000000
tied     0.765292
tying    0.643982
3-3      0.612363
2-2      0.594952
win      0.588898
1-1      0.587799
3-2      0.586701
match    0.567742
draw     0.563983
dtype: float64

In [15]:
dfn.dot(reject('tie', ['draw'], dfn)).sort_values(ascending = False).head(10)

0
tie          0.681923
tied         0.543408
tying        0.501142
jackets      0.402242
sweater      0.400014
knotted      0.399760
jacket       0.384109
necktie      0.382247
shoelaces    0.377615
31-31        0.376301
dtype: float64

In [16]:
find_most_similar('nail', dfn, 10)

0
nail        1.000000
nails       0.668212
biter       0.619956
remover     0.501642
tooth       0.498468
hammer      0.496885
glue        0.476576
polishes    0.469563
toe         0.462333
scissors    0.460371
dtype: float64

In [17]:
dfn.dot(reject('nails', ['fingernails'], dfn)).sort_values(ascending = False).head(10)

0
nails        0.652232
screws       0.466175
bolts        0.456011
metal        0.428832
nail         0.424728
cylinders    0.397965
reznor       0.383959
plastic      0.378701
stones       0.374274
inch         0.366976
dtype: float64

In [18]:
find_most_similar('java', dfn)

0
java          1.000000
sumatra       0.610773
semarang      0.603432
surabaya      0.596304
yogyakarta    0.571615
dtype: float64

In [19]:
dfn.dot(reject('java', ['sumatra'], dfn)).sort_values(ascending = False).head(10)

0
java               0.626956
software           0.441177
implementations    0.428661
desktop            0.423842
server             0.420261
javascript         0.412673
browser            0.410561
c++                0.408624
html               0.408063
functionality      0.405041
dtype: float64

In [20]:
find_most_similar('python', dfn, 10)

0
python         1.000000
monty          0.689272
spamalot       0.561178
cleese         0.545438
php            0.525527
pythons        0.507684
perl           0.499981
scripting      0.485102
skit           0.475383
reticulatus    0.470973
dtype: float64

In [21]:
dfn.dot(reject('python', ['monty'], dfn)).sort_values(ascending = False).head(10)

0
python         0.524904
php            0.414212
scripting      0.398441
java           0.336685
server-side    0.334762
pythons        0.323510
perl           0.322435
javascript     0.317562
c++            0.316876
bindings       0.316854
dtype: float64

In [22]:
dfn.dot(reject('python', ['monty', 'php'], dfn)).sort_values(ascending = False).head(10)

0
python         0.307225
pythons        0.263030
reticulated    0.236272
crocodile      0.232522
snake          0.230709
monkey         0.227266
burmese        0.225066
lizard         0.219475
turtles        0.216014
tortoise       0.214143
dtype: float64

In [23]:
find_most_similar("polish", dfn)

0
polish        1.000000
lithuanian    0.725827
hungarian     0.701218
poland        0.694825
slovak        0.667826
dtype: float64

In [24]:
dfn.dot(reject('polish', ['lithuanian'], dfn)).sort_values(ascending = False).head(10)

0
polish    0.473175
poland    0.383410
warsaw    0.344952
german    0.286086
jerzy     0.267660
walesa    0.263369
pope      0.262397
krakow    0.260597
lech      0.253146
iraqi     0.251096
dtype: float64

In [25]:
find_most_similar("poland", dfn)

0
poland       1.000000
hungary      0.718500
lithuania    0.697112
polish       0.694825
warsaw       0.676961
dtype: float64

# Removing gender

In [26]:
she_he = df.loc['she'] - df.loc['he']
she_he_len = she_he.dot(she_he)
def degenderize(row):
    return row - (row.dot(she_he)*she_he/she_he_len)

In [27]:
ungendered = df.apply(degenderize, axis=1)

In [28]:
ungendered = ungendered.div(np.sqrt((ungendered**2).sum(axis=1)), axis='index')

In [29]:
def similarity(a, b, data):
    return data.loc[a].dot(data.loc[b])

In [30]:
find_most_similar('jane', dfn, 10)

0
jane         1.000000
elizabeth    0.608416
anne         0.601765
mary         0.572299
sarah        0.570372
eliza        0.558833
alice        0.555983
helen        0.553562
ellen        0.553515
fonda        0.537337
dtype: float64

In [31]:
find_most_similar('jane', ungendered, 10)

0
jane         1.000000
elizabeth    0.556611
anne         0.545300
john         0.540406
bruce        0.537315
william      0.527928
henry        0.525371
eyre         0.515861
mary         0.515024
sarah        0.506965
dtype: float64

In [32]:
find_most_similar('mother', dfn, 10)

0
mother         1.000000
daughter       0.864802
wife           0.856802
grandmother    0.837379
husband        0.805565
sister         0.802924
father         0.793677
her            0.783749
daughters      0.758976
woman          0.757987
dtype: float64

In [33]:
find_most_similar('mother', ungendered, 10)

0
mother         1.000000
father         0.888926
daughter       0.842649
wife           0.837058
grandmother    0.811116
husband        0.804543
son            0.787408
sister         0.759808
brother        0.757006
her            0.738656
dtype: float64